# Text classification 

[Carlos Ortiz](https://www.linkedin.com/in/carlosortizdev/), [Sarah Santiago](https://www.linkedin.com/in/sarah-santiago-7a297b18a/), and [Vivek Datta](https://www.linkedin.com/in/vivek-datta/) did most of the coding. Jae Yeon Kim reviewed and modified the code slightly. Please use the `Python2` kernel to run this notebook. 

## Import libraries

In [1]:
#Include relevant imports here

from collections import Counter

# Data manipulation 
import numpy as np
import warnings
import pandas as pd
from pandas.api.types import CategoricalDtype
from sklearn.preprocessing import StandardScaler

# Data visualization 
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# ML
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, balanced_accuracy_score, cohen_kappa_score, precision_score, recall_score

warnings.filterwarnings('ignore')

## Import data 

In [2]:
# Read in scraped articles from csv file to dataframe
articles = pd.read_csv('/home/jae/ITS-Text-Classification/processed_data/cleaned_text.csv')

placebo_articles = pd.read_csv('/home/jae/ITS-Text-Classification/processed_data/placebo.csv')

## Feature extraction

In [3]:

# Conduct train-test split where 90% of data used for training

train, val = train_test_split(articles, 
                              test_size = 0.1, 
                              random_state = 42) # for reproducibility

## Exploratory data analysis

In [4]:
# Count all of the words that are in NA vs not in NA

NA_count = Counter(" ".join(train[train['category'] == 0]['text']).split())

not_NA_count = Counter(" ".join(train[train['category'] == 1]['text']).split())

# Creating dataframe named testing to hold these counts

testing = pd.DataFrame({"NA": NA_count, "Not NA": not_NA_count}).fillna(0)

testing.head()

,NA,Not NA
0,3.0,0.0
000,0.0,1.0
006,0.0,1.0
01,0.0,1.0
015,1.0,0.0


In [5]:
# Make column of times NA appeared more than not NA
testing['NA distinct'] = testing['NA'] - testing['Not NA']
testing = testing.sort_values(by = 'NA distinct', ascending = False)

# Make column of percentage of words
testing['NA percent'] = testing['NA'] / (testing['NA'] +  testing['Not NA'])
tester = testing[testing['Not NA'] > 0].sort_values(by = 'NA percent', ascending = False)

# Above work leads to creating of the top 100 words that appear in NA articles, as seen in words areray
words = tester[tester['NA'] > 100].sort_values(by = 'NA distinct', ascending = False).head(300).reset_index()['index'].values.astype(str)
words[:10]

array(['party', 'bjp', 'congress', 'minister', 'film', 'gandhi',
       'israeli', 'chief', 'says', 'films'], dtype='|S13')

## Feature engineering

In [6]:
# Create counts of how often the number of times top 100 words appeared in all articles, with 1 and 2 grams
cv = CountVectorizer(vocabulary=words, ngram_range = (1, 2))

# Encode the count vectorizer to create a dataframe holding counts
ohc_words_x_train = pd.SparseDataFrame(cv.fit_transform(train['text']), 
                       index = train.index,
                       columns = cv.get_feature_names(), 
                       default_fill_value=0)

ohc_words_x_train.head()

,party,bjp,congress,minister,film,gandhi,israeli,chief,says,films,...,states,world,people,muslims,muslim,community,us,arab,said,american
883,0,5,0,0,0,0,0,0,0,0,...,0,0,0,4,5,0,0,0,2,0
589,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,2,0,1,0,1
88,0,0,0,0,0,0,0,0,0,0,...,0,1,5,0,1,0,1,0,0,0
63,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,5,1,2,0,18,0
886,0,0,0,0,0,0,1,0,5,0,...,0,3,2,4,3,2,2,0,0,2


In [7]:
# Get addition features from one hot encoding the source, intervention, and group columns
features_x_train = pd.concat([pd.get_dummies(train[col]) for col in ['source', 'intervention', 'group']], axis=1)
features_x_train = features_x_train.drop(columns = ["The Arab American View"])
features_x_train.head()

,India - West,India Abroad,News India - Times,The Arab American News,post,pre,Arab,Indian
883,0,1,0,0,0,1,0,1
589,0,0,0,1,0,1,1,0
88,0,0,1,0,1,0,0,1
63,0,0,1,0,1,0,0,1
886,0,1,0,0,0,1,0,1


In [8]:
# Create final training matrix from the above two dataframes created from cells
xx_train = pd.merge(left=ohc_words_x_train, left_index=True
                  ,right=features_x_train, right_index=True,
                  how='inner')
xx_train.head()

,party,bjp,congress,minister,film,gandhi,israeli,chief,says,films,...,said,american,India - West,India Abroad,News India - Times,The Arab American News,post,pre,Arab,Indian
883,0,5,0,0,0,0,0,0,0,0,...,2,0,0,1,0,0,0,1,0,1
589,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,1,1,0
88,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,1
63,0,0,0,0,0,0,0,0,0,0,...,18,0,0,0,1,0,1,0,0,1
886,0,0,0,0,0,0,1,0,5,0,...,0,2,0,1,0,0,0,1,0,1


## Training and testing lasso (least absolute shrinkage and selection operator)

In [9]:

# Fit our Logistic Regression model with L1 regularization and determine the training accuracy

yy_train = train['category']

NA_model = LogisticRegressionCV(fit_intercept = True, 
                                penalty = 'l1', 
                                solver = 'saga')

NA_model.fit(xx_train, yy_train)

accuracy = NA_model.score(xx_train, yy_train)

print("Training Accuracy: ", accuracy)

('Training Accuracy: ', 0.8050383351588171)


In [10]:
# Repeat the above process for word count matrix in testing
cv = CountVectorizer(vocabulary=words, 
                     ngram_range = (1, 2))

ohc_words_x_test = pd.SparseDataFrame(cv.fit_transform(val['text']), 
                       val.index,
                       cv.get_feature_names(), 
                       default_fill_value=0)

ohc_words_x_test.head()

,party,bjp,congress,minister,film,gandhi,israeli,chief,says,films,...,states,world,people,muslims,muslim,community,us,arab,said,american
752,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,2,1,0,4,0
519,0,0,0,1,0,0,1,0,0,0,...,0,10,2,3,15,1,1,7,0,1
210,4,0,2,1,0,0,0,1,0,0,...,4,0,2,0,1,0,1,0,0,0
611,1,0,0,0,0,0,0,0,0,0,...,1,0,15,0,1,5,8,0,11,5
914,0,0,0,1,0,0,2,0,2,0,...,1,0,1,6,7,2,10,1,0,3


In [11]:

# Repeat the above process for feature matrix in testing 

features_x_test = pd.concat([pd.get_dummies(val[col]) for col in ['source', 'intervention', 'group']], axis=1)

features_x_test = features_x_test.drop(columns = ["The Arab American View"])

features_x_test.head()

,India - West,India Abroad,News India - Times,The Arab American News,post,pre,Arab,Indian
752,1,0,0,0,0,1,0,1
519,0,0,0,1,0,1,1,0
210,0,1,0,0,1,0,0,1
611,0,0,1,0,0,1,0,1
914,0,1,0,0,0,1,0,1


In [12]:

# Merging above dataframes to get the final testing matrix 

xx_test = pd.merge(left=ohc_words_x_test, left_index=True
                  ,right=features_x_test, right_index=True,
                   how='inner')

xx_test.head()

,party,bjp,congress,minister,film,gandhi,israeli,chief,says,films,...,said,american,India - West,India Abroad,News India - Times,The Arab American News,post,pre,Arab,Indian
752,0,0,0,0,0,0,0,0,0,0,...,4,0,1,0,0,0,0,1,0,1
519,0,0,0,1,0,0,1,0,0,0,...,0,1,0,0,0,1,0,1,1,0
210,4,0,2,1,0,0,0,1,0,0,...,0,0,0,1,0,0,1,0,0,1
611,1,0,0,0,0,0,0,0,0,0,...,11,5,0,0,1,0,0,1,0,1
914,0,0,0,1,0,0,2,0,2,0,...,0,3,0,1,0,0,0,1,0,1


In [13]:

def test_model(model, x_test, y_test):
    y_pred = model.predict(x_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    return(accuracy, precision, recall)


In [14]:
test_results = test_model(NA_model, x_test = xx_test, y_test = val['category'])


In [15]:
table = pd.DataFrame(test_results, columns = ['Scores'])

table['Metrics'] = ['Accuracy', 'Precision', 'Recall']

table

,Scores,Metrics
0,0.725490,Accuracy
1,0.753846,Precision
2,0.803279,Recall



## Label the unlabeled data 

In [16]:

# Get the unlabeled data and follow procedures in the beginning of notebook to clean

unlabeled_articles = pd.read_csv('/home/jae/ITS-Text-Classification/raw_data/unlabeled_articles.csv', index_col=0)

unlabeled_articles['date'] =  pd.to_datetime(unlabeled_articles['date'], format='%Y%m%d')

unlabeled_articles.head()

,text,source,date,intervention
1,Bleak Fate Of Women's Reservation Bill By SOU...,News India - Times,1998-07-24,post
2,Pak Christians Protest Blasphemy Law By Our S...,News India - Times,1998-07-24,post
3,Women's bill likely to be abandoned again ...,India Abroad,1998-07-24,post
4,Kashmir's Karma: An Analysis By HARI C. POLAV...,News India - Times,1998-07-17,post
5,Exiled Writer Can't See Her Dying Mother WASH...,News India - Times,1998-07-17,post


In [17]:
# Categorize whether article pertains to Arab-American or Indian-American issues based on publication source

response = []

for x in unlabeled_articles['source']:
    if x == "The Arab American News":
        response.append("Arab")
    else:
        response.append("Indian")

unlabeled_articles['group'] = response

unlabeled_articles.head()

,text,source,date,intervention,group
1,Bleak Fate Of Women's Reservation Bill By SOU...,News India - Times,1998-07-24,post,Indian
2,Pak Christians Protest Blasphemy Law By Our S...,News India - Times,1998-07-24,post,Indian
3,Women's bill likely to be abandoned again ...,India Abroad,1998-07-24,post,Indian
4,Kashmir's Karma: An Analysis By HARI C. POLAV...,News India - Times,1998-07-17,post,Indian
5,Exiled Writer Can't See Her Dying Mother WASH...,News India - Times,1998-07-17,post,Indian


In [18]:
# Create count encoding dataframe of top words again, except for unlabeled data
cv = CountVectorizer(vocabulary=words, ngram_range = (1, 2))

ohc_words_x_unlabeled = pd.SparseDataFrame(cv.fit_transform(unlabeled_articles['text']), 
                       unlabeled_articles.index,
                       cv.get_feature_names(), 
                       default_fill_value=0)

ohc_words_x_unlabeled.head()

,party,bjp,congress,minister,film,gandhi,israeli,chief,says,films,...,states,world,people,muslims,muslim,community,us,arab,said,american
1,9,6,7,0,0,1,0,2,0,0,...,0,0,0,0,1,0,0,0,1,0
2,0,0,0,1,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
3,17,6,5,5,0,2,0,0,0,0,...,1,0,0,0,1,0,0,0,5,0
4,0,0,0,1,0,0,0,0,0,0,...,13,0,3,2,4,0,1,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,2,0,1,0,2,0,1,0,2,0


In [19]:
# Same as above, except with the source, intervetion, and group features
features_x_unlabeled = pd.get_dummies(unlabeled_articles, columns=['source', 'intervention', 'group'])
features_x_unlabeled = features_x_unlabeled.drop(columns = ['text', 'date'])
features_x_unlabeled.head()

,source_India - West,source_India Abroad,source_News India - Times,source_The Arab American News,intervention_post,intervention_pre,group_Arab,group_Indian
1,0,0,1,0,1,0,0,1
2,0,0,1,0,1,0,0,1
3,0,1,0,0,1,0,0,1
4,0,0,1,0,1,0,0,1
5,0,0,1,0,1,0,0,1


In [20]:
# Data matrix created here
xx_unlabeled = pd.merge(left=ohc_words_x_unlabeled, left_index=True
                  ,right=features_x_unlabeled, right_index=True,
                  how='inner')

xx_unlabeled.head()

,party,bjp,congress,minister,film,gandhi,israeli,chief,says,films,...,said,american,source_India - West,source_India Abroad,source_News India - Times,source_The Arab American News,intervention_post,intervention_pre,group_Arab,group_Indian
1,9,6,7,0,0,1,0,2,0,0,...,1,0,0,0,1,0,1,0,0,1
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,1
3,17,6,5,5,0,2,0,0,0,0,...,5,0,0,1,0,0,1,0,0,1
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,1
5,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,1,0,1,0,0,1


In [21]:
# Use our model to predict whether the articles pertained to domestic vs non-domestic isues
values = NA_model.predict(xx_unlabeled)
values

array([1, 1, 0, ..., 1, 0, 1])


## Label the unlabeled placebo data 

In [22]:
ohc_words_placebo = pd.SparseDataFrame(cv.fit_transform(placebo_articles['text']), 
                       index = placebo_articles.index,
                       columns = cv.get_feature_names(), 
                       default_fill_value=0)

In [23]:
# Find the column names that only existed in the left dataframe 
lost_cols = list(set(xx_unlabeled.columns).difference(ohc_words_placebo.columns))

In [24]:
# For loop
for i in range(len(lost_cols)):
    
    ohc_words_placebo[lost_cols[i]] = 0


In [25]:
# Test 
set(xx_unlabeled.columns).difference(ohc_words_placebo.columns)

set()

In [26]:
# Predict 
placebo_values= NA_model.predict(ohc_words_placebo)

## Exporting files 

In [27]:
# Export results into a csv file
# pd.DataFrame(values, xx_unlabeled.index).to_csv("/home/jae/ITS-Text-Classification/processed_data/predicted.csv")

In [30]:
# Export results into a csv file 

pd.DataFrame(placebo_values).to_csv("/home/jae/ITS-Text-Classification/processed_data/placebo_predicted.csv")